In [1]:
import time
import numpy as np
import pandas as pd
from urllib.parse import unquote
from selenium import webdriver
from dateutil.parser import parse
from selenium.webdriver.support.ui import Select

In [2]:
class lost112Crawler:
    def __init__(self, startDate, endDate):
        self.startDate = startDate
        self.endDate = endDate
        self.sleepMin = 3
        self.sleepMax = 5
        
        self.getDriver()
        self.getStPage()
        self.sleep()
        self.getInfo()
        self.genCases()
                        
    def getInfo(self):
        self.categories = self.getCategories()
        self.locDvcd = self.getLocDvcd()
        self.rptDvcd = self.getRptDvcd()
        self.siteDvcd = self.getSiteDvcd()
    
    def setInfo(self, case):
        print('초기 세팅 시작')
        cat0, cat1, cat2, cat3, loc, rpt, site = case
        cat = cat0, cat1, cat2, cat3
        self.setLocDvcd(loc)
        self.setSiteDvcd(site)
        self.setRptDvcd(rpt)
        self.setCategory(cat)
        self.setDate(self.startDate, self.endDate)
        #self.sleep()
        self.driver.find_element_by_id('searchMain').click()
        print('초기 세팅 완료')
        
    def genCases(self):
        print('분류 케이스 생성 시작')
        rstList = []
        for _, cat in self.categories.iterrows():
            for loc in self.locDvcd:
                for rpt in self.rptDvcd:
                    for site in self.siteDvcd:
                        rst = cat[0], cat[1], cat[2], cat[3], loc, rpt, site
                        rstList.append(rst)
        df = pd.DataFrame(rstList, columns=['중분류코드','중분류명','소분류코드','소분류명','습득지역','접수구분','습득장소'])
        self.cases = df
        print('분류 케이스 생성 완료')
        
    def getDriver(self):
        print('드라이버 가져오기 시작')
        options = webdriver.ChromeOptions()
        #options.add_argument('headless')
        #options.add_argument('disable-gpu')
        self.driver = webdriver.Chrome('chromedriver.exe', options=options)
        print('드라이버 가져오기 완료')
        
    def sleep(self):
        return time.sleep(np.random.uniform(self.sleepMin, self.sleepMax))
        
    def getStPage(self):
        print('페이지 접근 시작')
        url = 'https://www.lost112.go.kr/find/findList.do'
        self.driver.get(url)
        print('페이지 접근 완료')

    def setDate(self, startDate, endDate):
        start_date = parse(startDate)
        end_date = parse(endDate)    
        self.driver.execute_script("showCalendar(this,'#CalendarControl', 'startYmdInput',70,30)")
        self.driver.execute_script('setCalendarControlDate({},{},{})'.format(start_date.year, start_date.month, start_date.day))
        self.driver.execute_script("showCalendar(this,'#CalendarControl', 'endYmdInput',70,30)")
        self.driver.execute_script('setCalendarControlDate({},{},{})'.format(end_date.year, end_date.month, end_date.day))

    def setLocDvcd(self, loc):
        if(loc not in self.locDvcd):
            print('습득지역을 올바르게 입력해주세요')
            return None
        wb = self.driver.find_element_by_id('fdLctCd')
        slt = Select(wb)
        slt.select_by_visible_text(loc)
        
    def setRptDvcd(self, rpt):
        if(rpt not in self.rptDvcd):
            print('접수구분을 올바르게 입력해주세요')
            return None
        wb = self.driver.find_element_by_id('site')
        slt = Select(wb)
        slt.select_by_visible_text(rpt)
        
    def setSiteDvcd(self, site):
        if(site not in self.siteDvcd):
            print('습득장소를 올바르게 입력해주세요')
            return None
        wb = self.driver.find_element_by_id('placeSeCd')
        slt = Select(wb)
        slt.select_by_visible_text(site)
        
    def setCategory(self, cat):
        self.driver.execute_script("fn_select_item('{}', '{}', '{}', '{}', '')".format(cat[0], cat[1], cat[2], cat[3]))

    def getTable(self):
        wb = self.driver.find_element_by_css_selector('table.type01')
        tableHtml = wb.get_attribute('outerHTML')
        df = pd.read_html(tableHtml)[0]
        return df

    def goTo(self, page):
        self.driver.execute_script('fn_find_link_page({})'.format(page))

    def getData(self, case):
        print('분류별 데이터 수집 시작')
        self.setInfo(case)
        self.sleep()
        rstList = []
        page = 1
        while(True):
            self.goTo(page)
            self.sleep()
            rst = self.getTable()
            rstList.append(rst)
            if len(rst)==0:
                break
            print('{:6} 페이지 데이터 수집'.format(page))
            page += 1
        df = pd.concat(rstList)
        if(len(df) == 0):
            print('수집된 데이터가 없습니다.')
            return None
        for idx in case.index[::-1]:
            df.insert(0, idx, case[idx])
        print('분류별 데이터 수집 완료')
        return df
    
    def getLocDvcd(self):
        print('습득지역 수집 시작')
        wb = self.driver.find_element_by_id('fdLctCd')
        slt = Select(wb)
        rstList = []
        for i in range(len(slt.options)-1):
            rst = slt.options[i+1].text
            rstList.append(rst)
        print('습득지역 수집 완료')
        return rstList
    
    def getSiteDvcd(self):
        print('습득장소 수집 시작')
        wb = self.driver.find_element_by_id('placeSeCd')
        slt = Select(wb)
        rstList = []
        for i in range(len(slt.options)-1):
            rst = slt.options[i+1].text
            rstList.append(rst)
        print('습득장소 수집 완료')
        return rstList
    
    def getRptDvcd(self):
        print('접수구분 수집 시작')
        wb = self.driver.find_element_by_id('site')
        slt = Select(wb)
        rstList = []
        for i in range(len(slt.options)-1):
            rst = slt.options[i+1].text
            rstList.append(rst)
        print('접수구분 수집 완료')
        return rstList
    
    def getCategory(self, webEle):
        url = unquote(webEle.find_element_by_tag_name('a').get_attribute('href'))
        splited = url.split('\'')
        return splited[1], splited[3], splited[5], splited[7]

    def getCategories(self):
        print('카테고리 수집 시작')
        wb = self.driver.find_element_by_id('find_kindPop')
        wbs = wb.find_elements_by_class_name('lost_group_sub_li')
        rstList = []
        for wb in wbs:
            rst = self.getCategory(wb)
            rstList.append(rst)
        df = pd.DataFrame(rstList, columns={'중분류코드','중분류명','소분류코드','소분류명'})
        print('카테고리 수집 완료')
        return df
    
    def close(self):
        self.driver.close()
        
    def to_excel(self, fileName):
        print('엑셀 추출 시작')
        writer = pd.ExcelWriter(fileName, 'xlsxwriter')
        self.data.to_excel(writer, index=False)
        writer.save()
        writer.close()
        print('엑셀 추출 완료')
        
    def run(self):
        print('데이터 수집 시작')
        rstList = []
        for i in range(len(self.cases)):
            rst = self.getData(self.cases.loc[i])
            rstList.append(rst)
        self.data = pd.concat(rstList)
        print('데이터 수집 완료')

In [3]:
startDate = '20181231'
endDate = '20181231'

lc = lost112Crawler(startDate, endDate)
lc.run()

드라이버 가져오기 시작
드라이버 가져오기 완료
페이지 접근 시작
페이지 접근 완료
카테고리 수집 시작
카테고리 수집 완료
습득지역 수집 시작
습득지역 수집 완료
접수구분 수집 시작
접수구분 수집 완료
습득장소 수집 시작
습득장소 수집 완료
분류 케이스 생성 시작
분류 케이스 생성 완료
데이터 수집 시작
분류별 데이터 수집 시작
초기 세팅 시작
초기 세팅 완료
수집된 데이터가 없습니다.
분류별 데이터 수집 시작
초기 세팅 시작
초기 세팅 완료
     1 페이지 데이터 수집
분류별 데이터 수집 완료
분류별 데이터 수집 시작
초기 세팅 시작
초기 세팅 완료
수집된 데이터가 없습니다.
분류별 데이터 수집 시작
초기 세팅 시작
초기 세팅 완료
수집된 데이터가 없습니다.
분류별 데이터 수집 시작
초기 세팅 시작
초기 세팅 완료
수집된 데이터가 없습니다.
데이터 수집 완료
